In [1]:
import numpy as np
import torch

In [2]:
weight = torch.tensor(2.0)
bias = torch.tensor(-1.0)

In [3]:
def forward(x):
    yhat = weight * x + bias
    return yhat

In [4]:
x = torch.tensor([1.0])
yhat = forward(x)
print(yhat)

tensor([1.])


In [5]:
x = torch.tensor([[1.0], [2.0], [3.0]])
print(forward(x))

tensor([[1.],
        [3.],
        [5.]])


### Linear Regression

In [2]:
from torch.nn import Linear
torch.manual_seed(101)

In [3]:
model = Linear(in_features=1, out_features=1, bias=True)
model

Linear(in_features=1, out_features=1, bias=True)

In [8]:
print(model.state_dict())
w = model.state_dict()['weight']
b = model.state_dict()['bias']
w, b

OrderedDict([('weight', tensor([[-0.6039]])), ('bias', tensor([-0.0994]))])


(tensor([[-0.6039]]), tensor([-0.0994]))

In [10]:
model.state_dict()['weight'][0] = torch.tensor(2)
model.state_dict()['bias'][0] = torch.tensor(-1)

model.state_dict()

OrderedDict([('weight', tensor([[2.]])), ('bias', tensor([-1.]))])

In [17]:
print(model.state_dict()['bias'][0], model.state_dict()['bias'])

tensor(-1.) tensor([-1.])


In [25]:
x = torch.tensor([[1.0]])
yhat = model(x)
print(yhat)

1.0


### 사용자 정의 모듈

In [26]:
from torch import nn

class LR(nn.Module):
    def __init__(self, input_size, output_size):
        super(LR, self).__init__() # LR, self 는 python 3 이상 부터는 옵셔널
        self.linear = nn.Linear(input_size, output_size)
        
    def forward(self, x):
        out = self.linear(x)
        return out

In [27]:
model = LR(1, 1)
model.state_dict()

OrderedDict([('linear.weight', tensor([[-0.8182]])),
             ('linear.bias', tensor([0.7744]))])

In [28]:
model.state_dict()['linear.weight'][0] = torch.tensor(2)
model.state_dict()['linear.bias'][0] = torch.tensor(-1)
model.state_dict()

OrderedDict([('linear.weight', tensor([[2.]])),
             ('linear.bias', tensor([-1.]))])

In [29]:
list(model.parameters())

[Parameter containing:
 tensor([[2.]], requires_grad=True),
 Parameter containing:
 tensor([-1.], requires_grad=True)]

In [31]:
x = torch.tensor([[1.0], [2.0], [3.0]])
yhat = model(x)
print(yhat)

tensor([[1.],
        [3.],
        [5.]], grad_fn=<AddmmBackward0>)


In [34]:
yhat.detach().numpy()

array([[1.],
       [3.],
       [5.]], dtype=float32)

### Backpropagation & 경사하강법

In [2]:
# 습도, 강수량, 최고온도, 최저온도
inputs = np.array([
    [73, 67, 43, 10],
    [91, 88, 64, 5],
    [87, 134, 58, 2],
    [102, 43, 37, 4],
    [69, 96, 70, 5]], dtype='float32')

# 사과 수확량, 오랜지 수확량
targets = np.array([
    [56, 70],
    [81, 101],
    [119, 133],
    [22, 37],
    [103, 119]], dtype='float32')

In [3]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs.size())
print(targets.size())

torch.Size([5, 4])
torch.Size([5, 2])


In [10]:
from torch import nn
from torch.nn import Linear
from torch import optim
torch.manual_seed(1)

In [11]:
model = Linear(in_features=4, out_features=2)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=1e-4)

In [12]:
for epoch in range(100):
    for x, y in zip(inputs, targets):
        optimizer.zero_grad()
        yhat = model(x)
        loss = criterion(yhat, y)
        loss.backward()
        optimizer.step()

In [13]:
preds = model(inputs)
preds

tensor([[ 58.0216,  69.7470],
        [ 83.1565,  98.7855],
        [119.9650, 128.4409],
        [ 21.8396,  34.9880],
        [102.8034, 117.8936]], grad_fn=<AddmmBackward0>)

In [14]:
loss = criterion(preds, targets)
loss

tensor(4.0759, grad_fn=<MseLossBackward0>)

### 사용자 정의 모델

In [15]:
class LinearReg(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearReg, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        
    def forward(self, x):
        yhat = self.fc(x)
        return yhat

In [17]:
model = LinearReg(inputs.shape[1], targets.shape[1])
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.00001)

In [18]:
print(model)
print(model.state_dict())
print(optimizer.state_dict())

LinearReg(
  (fc): Linear(in_features=4, out_features=2, bias=True)
)
OrderedDict([('fc.weight', tensor([[ 0.1387,  0.0247,  0.1826, -0.1949],
        [-0.0365, -0.0450,  0.0725, -0.0020]])), ('fc.bias', tensor([0.4371, 0.1556]))])
{'state': {}, 'param_groups': [{'lr': 1e-05, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'fused': None, 'params': [0, 1]}]}


In [37]:
for epoch in range(1000):
    for x, y in zip(inputs, targets):
        optimizer.zero_grad()
        yhat = model(x)
        loss = criterion(yhat, y)
        loss.backward()
        optimizer.step()

In [40]:
preds = model(inputs)
loss = criterion(preds, targets)
print(preds, targets, loss)
print(model.state_dict())

tensor([[ 55.9965,  70.0007],
        [ 82.5451, 100.6955],
        [119.2682, 132.9472],
        [ 21.6085,  37.0771],
        [102.1818, 119.1612]], grad_fn=<AddmmBackward0>) tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]]) tensor(0.3409, grad_fn=<MseLossBackward0>)
OrderedDict([('fc.weight', tensor([[-0.3949,  0.8401,  0.7048, -0.2424],
        [-0.3017,  0.7911,  0.9168, -0.0519]])), ('fc.bias', tensor([0.6497, 0.1153]))])


In [39]:
torch.save(model.state_dict(), 'model.pt')

In [ ]:
reload_model = LinearReg(inputs.shape[1], targets.shape[1])
reload_model.load_state_dict(torch.load('model.pt'))